In [66]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/Subject10_2.edf
/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/Subject26_1.edf
/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/Subject07_2.edf
/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/Subject27_2.edf
/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/Subject29_2.edf
/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/Subject13_1.edf
/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/SHA256SUMS.txt
/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/Subject35_1.edf
/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/Subject28_2.edf
/kaggle/input/eeg-dataset-during-arith

In [82]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import pandas as pd

# Load EEG data from CSV file
file_path = '/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/subject-info.csv'
eeg_data = pd.read_csv(file_path)

# Split data into features (X) and target (y)
X = eeg_data.drop(columns=['Subject', 'Count quality'])  # Assuming 'Subject' and 'Count quality' are not features
y = eeg_data['Count quality']

# Define numeric and categorical features
numeric_features = ['Age', 'Recording year', 'Number of subtractions']
categorical_features = ['Gender']

# Preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [68]:
# Check the structure and columns of the loaded data
print(df.head())

     Subject  Age Gender  Recording year  Number of subtractions  \
0  Subject00   21      F            2011                    9.70   
1  Subject01   18      F            2011                   29.35   
2  Subject02   19      F            2012                   12.88   
3  Subject03   17      F            2010                   31.00   
4  Subject04   17      F            2010                    8.60   

   Count quality  
0              0  
1              1  
2              1  
3              1  
4              0  


In [90]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Fit preprocessor on training data and transform it
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Define the neural network model
model = Sequential([
    Dense(64, activation='relu', input_dim=X_train_processed.shape[1]),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model
history = model.fit(X_train_processed, y_train, epochs=50, batch_size=32, validation_split=0.2, 
                    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])

Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6818 - loss: 0.6564 - val_accuracy: 0.5000 - val_loss: 0.7112
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.5909 - loss: 0.6626 - val_accuracy: 0.5000 - val_loss: 0.7055
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.5000 - loss: 0.7066 - val_accuracy: 0.5000 - val_loss: 0.6997
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.5455 - loss: 0.7342 - val_accuracy: 0.5000 - val_loss: 0.6941
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.4091 - loss: 0.7130 - val_accuracy: 0.5000 - val_loss: 0.6883
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.6364 - loss: 0.6584 - val_accuracy: 0.3333 - val_loss: 0.6823
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.7273 - loss: 0.5897 - val_accuracy: 0.5000 - val_loss: 0.6762
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.6818 - loss: 0.6307 - val_accuracy: 0.5000 - val_loss: 0.6701
Epoch 9/50
1/

In [91]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_processed, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.8750 - loss: 0.4832
Test Accuracy: 87.50%
